In [54]:
import numpy as np
from ppopt.mplp_program import MPLP_Program
from ppopt.mpmodel import MPModeler
from ppopt.mp_solvers.solve_mpqp import solve_mpqp, mpqp_algorithm
from ppopt.plot import parametric_plot
from numpy.polynomial.legendre import leggauss
from scipy.optimize import linprog
from typing import Union
from collections import defaultdict
from itertools import product
import time

In [55]:
def gauss_legendre_between_bounds(expr_coeffs: np.ndarray, n_gl: int, max_idx: int = 0, min_idx: int = 1):
    """
    Generate n Gauss–Legendre quadrature points and weights between min and max bounds
    defined by two linear expressions.

    Parameters:
        expr_coeffs (np.ndarray): 2xD array. Row 0 = max point co`efficients, Row 1 = min.
        n (int): Number of quadrature points.

    Returns:
        points (np.ndarray): (n, D) array of quadrature points.
        weights (np.ndarray): (n,) array of weights.
    """
    if expr_coeffs.shape[0] != 2:
        raise ValueError("expr_coeffs must have two rows")

    max_coeffs = expr_coeffs[max_idx]
    min_coeffs = expr_coeffs[min_idx]

    # Get Gauss–Legendre points and weights on [-1, 1]
    nodes, weights = leggauss(n_gl)
    weights = weights.reshape(-1,1)
    
    # Affine transformation to domain [min_coeffs, max_coeffs]
    points = 0.5 * (np.outer((nodes + 1), max_coeffs) + np.outer((1 - nodes), min_coeffs))

    # Adjust weights to match new domain
    weights = 0.5 * weights@(max_coeffs - min_coeffs).reshape(1,-1)

    return points, weights

In [56]:
def get_quadrature_points(solution, nq: int, t_vector: np.ndarray):
    # Augment t_vector once
    t_vector_aug = np.append(t_vector, 1).reshape(-1, 1)
    
    if isinstance(solution, list):
        qpoints, qweights = np.polynomial.legendre.leggauss(nq)
        min, max = solution[0], solution[1]
        qps_mapped = 0.5*(max*(1+qpoints) + min*(1-qpoints))
        qws_mapped = 0.5*(max-min)*qweights
        return max, min, qps_mapped, qws_mapped
    
    for region in solution.critical_regions:
        if region.is_inside(t_vector):
            coeffs = np.concatenate([region.A, region.b], axis=1)[:2, :]
            qpoints, qweights = gauss_legendre_between_bounds(expr_coeffs=coeffs, n_gl=nq)
            return coeffs[0], coeffs[1], qpoints, qweights
    
    # print(f't_vector: {t_vector}')
    # print(f'solution:{solution}')
    raise ValueError("No region found that contains the given t_vector.")


In [57]:
def mpformulate_theta_bounds(flex_sol, num_theta:int, theta_bounds:list, num_design:int=0, design_bounds:list=None, psi_idx:int=0, theta_m:int=0):
    A0, b0, F0 = np.empty((len(flex_sol), num_theta)), np.empty((len(flex_sol), 1)), np.empty((len(flex_sol), num_design))
    num_cr = len(flex_sol.critical_regions)
    for i, region in enumerate(flex_sol.critical_regions):
        A0[i] = region.A[psi_idx,:num_theta]
        b0[i] = -region.b[psi_idx]
        F0[i] = -region.A[psi_idx, num_theta:num_theta+num_design]
    # print(f'num_cr:{num_cr}')
    # print(f'num_theta:{num_theta}')
    # print(f'num_design:{num_design}')
    # print(f"A0: {A0}")
    # print(f"b0: {b0}")
    # print(f"F0: {F0}")
    
    c = np.hstack([np.array([-1, 1]).reshape(1, -1), np.zeros((1, 2 * (num_theta - 1 - theta_m)))]).reshape(-1,1)
    # print(f'c:{c}')
    # print(f'c.shape: {c.shape}')
    
    row1_block = np.hstack([block for i in range(theta_m, num_theta) for block in (A0[:, [i]], np.zeros((num_cr, 1)))])
    row2_block = np.hstack([block for i in range(theta_m, num_theta) for block in (np.zeros((num_cr, 1)), A0[:, [i]])])
    bound_row = np.hstack([np.array([-1, 1]).reshape(1, -1), np.zeros((1, 2 * (num_theta - 1 - theta_m)))])
    A = np.vstack([row1_block, row2_block, bound_row, -np.eye(2*(num_theta-theta_m)), np.eye(2*(num_theta-theta_m))])
    # print(f'A: {A}')
    # print(f'A.shape: {A.shape}')
    
    x_lb = np.array([val for i in range(theta_m, len(theta_bounds)) for val in [theta_bounds[i][0]] * 2])
    x_ub = np.array([val for i in range(theta_m, len(theta_bounds)) for val in [theta_bounds[i][1]] * 2])
    b = np.vstack([b0, b0, np.zeros((1,1)), -x_lb.reshape(-1,1), x_ub.reshape(-1,1)])
    # print(f'b: {b}')
    # print(f'b.shape: {b.shape}')
    
    if F0.size==0 and theta_m==0:
        # print('here')
        return A, b, c, np.array([]), np.array([]), np.array([]), np.array([]) 
    
    F = np.vstack([F0, F0, np.zeros((1,num_design)), np.zeros((4*(num_theta-theta_m), num_design))]) if num_design>0 else np.vstack([F0, F0])
    # print(f'F:{F}')
    # print(f'F.shape: {F.shape}')
    if theta_m > 0:
        F_lltheta = np.hstack([A0[:, [i]] for i in range(theta_m)])
        # print(f'F_lltheta: {F_lltheta}')
        # print(f'F_lltheta.shape: {F_lltheta.shape}')
        F = np.hstack([np.vstack([-F_lltheta, -F_lltheta, np.zeros((1,len(range(theta_m)))), np.zeros((4*(num_theta-theta_m), theta_m))]), F]) if F.size > 0 else np.vstack([-F_lltheta, -F_lltheta, np.zeros((1,len(range(theta_m)))), np.zeros((4*(num_theta-theta_m), theta_m))])
    # print(f'F:{F}')
    # print(f'F.shape: {F.shape}')
    
    H = np.zeros((2*(num_theta-theta_m), theta_m+num_design))
    # print(f'H:{H}')
    # print(f'H.shape: {H.shape}')
    
    A_t = np.vstack([-np.eye(theta_m+num_design), np.eye(theta_m+num_design)])
    # print(f'A_t:{A_t}')
    # print(f'A_t.shape: {A_t.shape}')
    
    theta_lb = np.array([-theta_bounds[i][0] for i in range(theta_m)] + ([-j[0] for j in design_bounds] if isinstance(design_bounds, list) 
                                                                        else [])).reshape(-1, 1)
    theta_ub = np.array([theta_bounds[i][1] for i in range(theta_m)] + ([j[1] for j in design_bounds] if isinstance(design_bounds, list) 
                                                                        else [])).reshape(-1, 1)
    
    b_t = np.vstack([theta_lb, theta_ub])
    # print(f'b_t:{b_t}')
    # print(f'b_t.shape: {b_t.shape}')
    
    return A, b, c, H, A_t, b_t, F

In [58]:
def get_theta_bounds(flex_sol, numt, tbounds, numd:int=0, dbounds:list=None):

    theta_bound_dict = defaultdict(dict)
    prob_dict = defaultdict(dict)
    for i in range(numt):
        A, b, c, H, A_t, b_t, F = mpformulate_theta_bounds(flex_sol=flex_sol, num_theta=numt ,num_design=numd, theta_bounds=tbounds, design_bounds=dbounds, theta_m=i)
        if F.size != 0:
            prob = MPLP_Program(A=A, b=b, c=c, H=H, A_t=A_t, b_t=b_t, F=F)
            prob.process_constraints()
            solution = solve_mpqp(problem=prob, algorithm=mpqp_algorithm.geometric)
            prob_dict[f't{i}'] = prob
            theta_bound_dict[f't{i}'] = solution
        else:
            linsol = linprog(c=c, A_ub=A, b_ub=b)
            prob_dict[f't{i}'] = linsol
            theta_bound_dict[f't{i}'] = [linsol.x[1], linsol.x[0]]
            # if linsol.success:
                # print("Optimal value:", linsol.fun)
                # print("Optimal x:", linsol.x)
        print(f'Finished solving for theta{i+1}')
    probs = [p for key, p in prob_dict.items()]
    sols = [sol for key, sol in theta_bound_dict.items()]

    return probs, sols

In [59]:
# # Straub and Grossmann (1990) Example 2
# t_bounds=[(0,80),(0,80)]
# d_bounds=[(0,10), (0,10)]
# d_vector = np.array([1,8])
# nt = len(t_bounds)
# nd = len(d_bounds)
# m = MPModeler()
# 
# u = m.add_var(name='u')
# z = m.add_var(name='z')
# 
# t1 = m.add_param(name='t1')
# t2 = m.add_param(name='t2')
# d1 = m.add_param(name='d1')
# d2 = m.add_param(name='d2')
# m.add_constr(z - 1.6*(t1-20) - 0.6*(t2-20) + d1 - 14 <= u)
# m.add_constr(z - 0.85*(t1-20) - 0.925*(t2-20) + d1 + d2 -20 <= u)
# m.add_constr(z - 1.1*(t1-20) - 1.4*(t2-20) + d2 - 8 <= u)
# m.add_constr(-z + (t1-20) + (t2-20) + d1 + d2 - 7 <= u)
# m.add_constr(t_bounds[0][0] <= t1)
# m.add_constr(t_bounds[1][0] <= t2)
# m.add_constr(d_bounds[0][0] <= d1)
# m.add_constr(d_bounds[1][0] <= d2)
# m.add_constr(t1 <= t_bounds[0][1])
# m.add_constr(t2 <= t_bounds[1][1])
# m.add_constr(d1 <= d_bounds[0][1])
# m.add_constr(d2 <= d_bounds[1][1])
# m.set_objective(u)
# prob = m.formulate_problem()
# prob.process_constraints()
# solution_flexibility = solve_mpqp(problem=prob, algorithm=mpqp_algorithm.geometric)
# 
# def joint_pdf(theta:list):
#     return (1/628.32)*np.exp((-(theta[0]-40)**2 - (theta[1]-40)**2)/200)

In [60]:
# Bansal (2000) Illustrative Example
t_bounds=[(0,4),(0,4)]
d_bounds=[(0,5), (0,5)]
d_vector = np.array([3.2, 0.4])
nt = len(t_bounds)
nd = len(d_bounds)

m = MPModeler()

u = m.add_var(name='u')
x = m.add_var(name='x')
z = m.add_var(name='z')

t1 = m.add_param(name='t1')
t2 = m.add_param(name='t2')
d1 = m.add_param(name='d1')
d2 = m.add_param(name='d2')
m.add_constr(2*x - 3*z + t1 - d2 == 0)
m.add_constr(x - z/2 -t1/2 +t2/2 +d1 -7*d2/2 <= u)
m.add_constr(-2*x +2*z -4*t1/3 -t2 +2*d2 +1/3<= u)
m.add_constr(-x + 5*z/2 +t1/2 -t2 -d1 +d2/2 -1 <= u)
m.add_constr(-50 <= x)
m.add_constr(-50 <= z)
m.add_constr(t_bounds[0][0] <= t1)
m.add_constr(t_bounds[1][0] <= t2)
m.add_constr(d_bounds[0][0] <= d1)
m.add_constr(d_bounds[1][0] <= d2)
m.add_constr(t1 <= t_bounds[0][1])
m.add_constr(t2 <= t_bounds[1][1])
m.add_constr(d1 <= d_bounds[0][1])
m.add_constr(d2 <= d_bounds[1][1])
m.set_objective(u)
prob = m.formulate_problem()
prob.process_constraints()
solution_flexibility = solve_mpqp(problem=prob, algorithm=mpqp_algorithm.geometric)

def joint_pdf(theta:list):
    return (2/np.pi)*np.exp(-2*((theta[0]-2)**2 + (theta[1]-2)**2))

Using a found active set [0, 2, 3]


In [61]:
for i in range(len(solution_flexibility)):
    print(f'i:{i}\nA:{solution_flexibility.critical_regions[i].A}\nb:{solution_flexibility.critical_regions[i].b}')

i:0
A:[[ 3.33333333e-01 -1.00000000e+00 -5.00000000e-01  5.00000000e-01]
 [-1.50000000e+00  1.00310391e-16  7.50000000e-01  1.25000000e+00]
 [-6.66666667e-01 -2.02168971e-16  5.00000000e-01  5.00000000e-01]]
b:[[-0.33333333]
 [ 1.        ]
 [ 0.66666667]]
i:1
A:[[-6.66666667e-01 -2.50000000e-01  5.00000000e-01 -1.00000000e+00]
 [-2.71684799e-16 -1.12500000e+00 -7.50000000e-01  3.50000000e+00]
 [ 3.33333333e-01 -7.50000000e-01 -5.00000000e-01  2.00000000e+00]]
b:[[0.16666667]
 [0.25      ]
 [0.16666667]]


In [62]:
probs, sols = get_theta_bounds(flex_sol=solution_flexibility, numt=nt, numd=nd, tbounds=t_bounds, dbounds=d_bounds)

Using a found active set [6, 9, 11, 12]
Finished solving for theta1
Using a found active set [2, 7]
Finished solving for theta2


In [63]:
nq = 5

In [64]:
t0_vector = np.block([np.array([]), d_vector]) if isinstance(d_vector, np.ndarray) else np.array([])

In [65]:
_, _, t0_points, t0_weights = get_quadrature_points(solution=sols[0], nq=nq, t_vector=t0_vector)

In [66]:
t0_points

array([[ 0.71481744, -1.42963488, -1.0037221 ],
       [ 0.57692599, -1.15385198, -0.03848194],
       [ 0.375     , -0.75      ,  1.375     ],
       [ 0.17307401, -0.34614802,  2.78848194],
       [ 0.03518256, -0.07036512,  3.7537221 ]])

In [67]:
t0_weights

array([[-0.08884758,  0.17769516,  0.62193307],
       [-0.17948575,  0.3589715 ,  1.25640026],
       [-0.21333333,  0.42666667,  1.49333333],
       [-0.17948575,  0.3589715 ,  1.25640026],
       [-0.08884758,  0.17769516,  0.62193307]])

In [68]:
t0_points @ np.append(d_vector, 1).reshape(-1, 1)

array([[0.71183977],
       [1.34614044],
       [2.275     ],
       [3.20385956],
       [3.83816023]])

In [69]:
t0_weights @ np.append(d_vector, 1).reshape(-1,1)

array([[0.40869888],
       [0.82563446],
       [0.98133333],
       [0.82563446],
       [0.40869888]])

In [70]:
sols[0]

Solution(program=<ppopt.mplp_program.MPLP_Program object at 0x000001CF3637FA30>, critical_regions=[Critical region with active set [6, 9, 11, 12]
The Omega Constraint indices are [0, 1, 2, 3]
The Lagrange multipliers Constraint indices are []
The Regular Constraint indices are [[3], [3]]
  x(θ) = Aθ + b 
 λ(θ) = Cθ + d 
  Eθ <= f
 A = [[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]] 
 b = [[4.]
 [0.]
 [4.]
 [4.]] 
 C = [[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]] 
 d = [[1.]
 [1.]
 [0.]
 [0.]] 
 E = [[ 0.4472136  -0.89442719]
 [-1.         -0.        ]
 [-0.         -1.        ]
 [ 1.          0.        ]
 [ 0.          1.        ]] 
 f = [[0.74535599]
 [0.        ]
 [0.        ]
 [5.        ]
 [5.        ]], Critical region with active set [3, 9, 11, 12]
The Omega Constraint indices are [1, 2]
The Lagrange multipliers Constraint indices are []
The Regular Constraint indices are [[5], [6]]
  x(θ) = Aθ + b 
 λ(θ) = Cθ + d 
  Eθ <= f
 A = [[ 0.00000000e+00  0.00000000e+00]
 [ 7.50000000e-01 -1.50000000e+00]

In [71]:
sols[0].get_region(np.array([1,8]))

In [72]:
# def reduce_all_x_to_d(A_list, b_list, m):
#     """
#     Reduces all x1 to xn to be functions of d1 to dm.
# 
#     Args:
#         A_list: list of 1D np.arrays, A_list[i] is for x_{i+1} and has shape (i + m,)
#         b_list: list of floats, b_list[i] is the constant for x_{i+1}
#         m: number of d variables
# 
#     Returns:
#         A_out: list of np.arrays of shape (m,) — coefficients for d in x_i
#         b_out: list of scalars — constants in x_i
#     """
#     n = len(A_list)
#     A_out = []
#     b_out = []
# 
#     for i in range(n):
#         Ai = A_list[i]
#         bi = b_list[i]
# 
#         if i == 0:
#             # x1 = Ai[-m:] * d + bi
#             coeff_d = Ai[-m:] if m > 0 else np.array([])
#             b = bi
#         else:
#             coeff_prev_x = Ai[:i]            # Coefficients of x1 to x_i
#             coeff_d_i = Ai[-m:] if m > 0 else np.array([])
#             b = bi
# 
#             coeff_d = np.zeros(m)
# 
#             for j in range(i):
#                 cj_d = A_out[j]   # coeffs of d in x_{j+1}
#                 cj_b = b_out[j]   # constant term in x_{j+1}
# 
#                 coeff_d += coeff_prev_x[j] * cj_d
#                 b += coeff_prev_x[j] * cj_b
# 
#             coeff_d += coeff_d_i
# 
#         A_out.append(coeff_d)
#         b_out.append(b)
# 
#     return A_out, b_out

# A1 = np.array([2.0, 3.0])                           # x1 = 2d1 + 3d2
# A2 = np.array([-1.0, 4.0, -2.0])                    # x2 = -1*x1 + 4*d1 -2*d2
# A3 = np.array([0.5, 2.0, -1.0, 0.0])                # x3 = 0.5*x1 + 2*x2 -1*d1 + 0*d2
# 
# A_list = [A1, A2, A3]
# b_list = [1.0, 0.5, 1.0]
# m = 2  # two d variables
# 
# A_out, b_out = reduce_all_x_to_d(A_list, b_list, m)
# 
# for i, (Ai, bi) in enumerate(zip(A_out, b_out), 1):
#     print(f"x{i} = {Ai[0]} * d1 + {Ai[1]} * d2 + {bi}")

In [73]:
theta_max_min_dict = defaultdict(dict)
for n_s, s in enumerate(sols):
    for n_cr, cr in enumerate(s.critical_regions):
        theta_max_min_dict[n_s][n_cr] = np.concatenate([cr.A, cr.b], axis=1)[:2, :]

In [74]:
theta_max_min_dict

defaultdict(dict,
            {0: {0: array([[0., 0., 4.],
                     [0., 0., 0.]]),
              1: array([[ 0.  ,  0.  ,  4.  ],
                     [ 0.75, -1.5 , -1.25]])},
             1: {0: array([[ 0.        ,  0.        ,  0.        ,  4.        ],
                     [ 0.33333333, -0.5       ,  0.5       , -0.33333333]]),
              1: array([[0., 0., 0., 4.],
                     [0., 0., 0., 0.]]),
              2: array([[ 0.        ,  0.        ,  0.        ,  4.        ],
                     [-2.66666667,  2.        , -4.        ,  0.66666667]])}})

In [75]:
theta_max_min_dict[0][1][0]

array([0., 0., 4.])

In [76]:
po, we = leggauss(5)

In [77]:
theta_qpoints = defaultdict(dict)

for s in theta_max_min_dict:
    for cr in theta_max_min_dict[s]:
        theta_qpoints[s][cr] = 0.5 * (np.outer((po+1), theta_max_min_dict[s][cr][0]) + np.outer((1-po),  theta_max_min_dict[s][cr][1]))

In [78]:
theta_qpoints

defaultdict(dict,
            {0: {0: array([[0.        , 0.        , 0.18764031],
                     [0.        , 0.        , 0.92306138],
                     [0.        , 0.        , 2.        ],
                     [0.        , 0.        , 3.07693862],
                     [0.        , 0.        , 3.81235969]]),
              1: array([[ 0.71481744, -1.42963488, -1.0037221 ],
                     [ 0.57692599, -1.15385198, -0.03848194],
                     [ 0.375     , -0.75      ,  1.375     ],
                     [ 0.17307401, -0.34614802,  2.78848194],
                     [ 0.03518256, -0.07036512,  3.7537221 ]])},
             1: {0: array([[ 0.31769664, -0.47654496,  0.47654496, -0.13005633],
                     [ 0.25641155, -0.38461733,  0.38461733,  0.66664983],
                     [ 0.16666667, -0.25      ,  0.25      ,  1.83333333],
                     [ 0.07692178, -0.11538267,  0.11538267,  3.00001684],
                     [ 0.01563669, -0.02345504,  0.023455

In [79]:
we

array([0.23692689, 0.47862867, 0.56888889, 0.47862867, 0.23692689])

In [80]:
len(sols[0])

2

In [81]:
len(sols[1])

3